## 1.1 将序列分解为单独的变量
### 1.1.1 问题
有一个包含N个元素的元组或序列，将它分解为N个单独的变量
### 1.12 解决方案
任何序列(或可迭代对象)都可以通过一个就按单的赋值操作来分解为单独的变量。唯一的要求是变量的总书和结构要与序列相吻合。

In [2]:
p = (4, 5)
x, y = p
print(x)
print(y)

4
5


In [3]:
data = ['ACME', 50, 91.1, (2012, 12, 21)]
name, shares, price, date = data
print(name)
print(shares)
print(price)
print(date)

ACME
50
91.1
(2012, 12, 21)


In [4]:
data = ['ACME', 50, 91.1, (2012, 12, 21)]
name, shares, price, (year, mon, day) = data
print(year)
print(mon)
print(day)

2012
12
21


如果元素的数量不匹配，将得到一个错误提示。

In [2]:
p = (4, 5)
x, y, z = p

ValueError: not enough values to unpack (expected 3, got 2)

### 1.1.3 讨论
只要对象是可迭代的，就可以执行分解操作。包括字符串、文件、迭代器以及生成器。

In [4]:
s = 'Hello'
a, b, c, d, e = s
print(a, b, c, d, e)

H e l l o


做分解操作时，可以丢弃某些特定的值。可以选一个用不到的变量名如`_`，以此来作为要丢弃的值的名称。

In [5]:
data = ['ACME', 50, 91.1, (2012, 12, 21)]
_, shares, price, _ = data
print(shares)
print(price)

50
91.1


## 1.2 从任意长度的可迭代对象中分解元素
### 1.2.1 问题
需要从某个可迭代对象中分解出N个元素，但是这个可迭代对象的长度可能超过N，这回导致出现“分解的值过多(too many values to unpack)”的异常
### 1.2.2 解决方案
Python的`*`表达式可以用来解决这个问题。

In [6]:
# 期末作业成绩等于去掉第一个和最后一个，只对中间剩下的成绩做平均分统计。
from audioop import avg


def drop_first_last(grades):
    first, *middle, last = grades
    return avg(middle)

In [7]:
record = ('Charlie', 'Charlie@gmail.com', '773-555-1212', '847-555-1212')
name, email, *phone_numbers = record
print(name, email, phone_numbers)

Charlie Charlie@gmail.com ['773-555-1212', '847-555-1212']


### 1.2.3 讨论
对于分解未知或任意长度的可迭代对象，这种扩展的分解操作非常有用。通常，这类可迭代独享中会有一些已知的组件或模式(例如，元素1之后的所有内容都是电话号码)。利用*表达式可以分解可迭代对象可以轻松利用这些模式。

In [8]:
records = [
    ('foo', 1, 2),
    ('bar', 'hello'),
    ('foo', 3, 4),
]

def do_foo(x, y):
    print('foo', x, y)

def do_bar(s):
    print('bar', s)

for tag, *args in records:
    if tag == 'foo':
        do_foo(*args)
    elif tag == 'bar':
        do_bar(*args)

foo 1 2
bar hello
foo 3 4


当和某些特定的字符串处理操作相结合，比如做拆分(splitting)操作时，这种*式的语法所支持的分解操作也非常有用。

In [9]:
line = 'nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/flase'
uname, *fields, homedir, sh = line.split(':')
print(uname, homedir, sh)

nobody /var/empty /usr/bin/flase


In [10]:
record = ('ACME', 50, 123.45, (12, 18, 2012))
name, *_, (*_, year) = record
print(name, year)

ACME 2012


## 1.3 保存最后N个元素
### 1.3.1 问题
在迭代或其他形式的处理过程中对最后几项操作做一个有限的历史记录统计
### 1.3.2 解决方案
保存有限的历史记录是`collections.deque`的强项。以下代码对一些列文本行做简单的文本匹配操作，当发现有匹配时就输出当前的匹配行以及最后检查过的N行文本

deque(maxlen=N)创建了一个固定长度的队列。当有新纪录加入而队列已满时会自动移除最老的那条记录。

In [12]:
from collections import deque
q = deque(maxlen=3)
q.append(1)
q.append(2)
q.append(3)
q

deque([1, 2, 3])

In [13]:
q.append(4)
q

deque([2, 3, 4])

In [14]:
q.append(5)
q

deque([3, 4, 5])

当需要一个简单的队列结构时，deque很有帮助。如果不指定队列的大小，就得到了一个无界限的队列，可以在两端执行添加和弹出操作

In [20]:
q = deque()
q.append(1)
q.append(2)
q.append(3)
q

deque([1, 2, 3])

In [21]:
q.appendleft(4)
q

deque([4, 1, 2, 3])

In [22]:
q.pop()
q

deque([4, 1, 2])

In [23]:
q.popleft() # 返回弹出的值

4

读取文件`somefile.txt`，将含有单词`Python`的行输出

In [6]:
from collections import deque

def search(lines, pattern, history=5):
    previous_lines = deque(maxlen=history)
    for line in lines:
        if pattern in line:
            yield line, previous_lines  # 当检测到时，输出当前行line，以及包含前几行的previous_lines
        previous_lines.append(line)

if __name__ == '__main__':
    with open('src/somefile.txt') as f:
        for line, prelines in search(f, 'Python', 5):
            for pline in prelines:
                print(pline, end='')    # 输出记录的前几行，最多保存5行
            print(line, end='') # 输出当前行
            print('-'*20)

hello world
I like Python
--------------------
hello world
I like Python
Life is short, I use Python
--------------------
I like Python
Life is short, I use Python
However, I'm learning cpp now
I love China
one piece
Python is easy to learn
--------------------
Life is short, I use Python
However, I'm learning cpp now
I love China
one piece
Python is easy to learn
I like Python
--------------------


## 1.4 找出最大或最小的N个元素
### 1.4.1 解决方案
`heapq`模块中有两个函数——`nlargest()`和`nsmallest()`

In [7]:
import heapq
nums = [1, 8, 2, 23, 7, -4, 18, 23, 42, 37, 2]
print(heapq.nlargest(3, nums))  # 输出列表nums中最大的3个元素 [42, 37, 23]
print(heapq.nsmallest(3, nums)) # 输出列表nums中最小的3个元素 [-4, 1, 2]

[42, 37, 23]
[-4, 1, 2]


这两个函数都可以接受一个参数key，运行工作在更加复杂的数据结构之上，自定义排序

In [8]:
portfolio = [
    {'name':'IBM', 'shares':100, 'price':91.1},
    {'name':'AAPL', 'shares':50, 'price':543.22},
    {'name':'FB', 'shares':200, 'price':21.09},
    {'name':'HPQ', 'shares':35, 'price':31.75},
    {'name':'YHOO', 'shares':45, 'price':16.35},
    {'name':'ACME', 'shares':75, 'price':115.65}
]

cheap = heapq.nsmallest(3, portfolio, key=lambda s:s['price'])  # 按'price'排序最小的3个元素
cheap

[{'name': 'YHOO', 'shares': 45, 'price': 16.35},
 {'name': 'FB', 'shares': 200, 'price': 21.09},
 {'name': 'HPQ', 'shares': 35, 'price': 31.75}]

### 1.4.2 讨论
如果正在寻找最大或最小的N个元素，且同集合中元素的总数目相比，N很小，以下方法可以提供更好的性能。这些函数首先在底层将数据转化成列表，且元素会以堆的顺序排列。

堆最重要的特性就是heap[0]总是最小那个的元素。接下来的元素可依次通过`heapq.heappop()`方法找到，该方法会将第一个元素(最小的)弹出，然后以第二小的元素取而代之(复杂度是O(logN)，N代表堆的大小)。要找到第3小的元素，可以这样做：

In [10]:
nums = [1, 8, 2, 23, 7, -4, 18, 23, 42, 37, 2]
import heapq
heap = list(nums)
heapq.heapify(heap) # heapq.heapify()将列表(list)转换为堆(heap)
heap

[-4, 2, 1, 23, 7, 2, 18, 23, 42, 37, 8]

In [12]:
heapq.heappop(heap) # 最小的元素

-4

In [13]:
heapq.heappop(heap) # 第2小的元素

1

In [14]:
heapq.heappop(heap) # 第3小的元素

2

- 当所要找的元素数量相对较小时，函数`nlargest()`和`nsmallest()`才是最适用的。
- 如果只是简单地找到最小或最大地元素(N=1)，用`min()`和`max()`会更加快。
- 如果N和集合本身地大小差不多大，通常更快地方法是先对集合排序，然后做切片操作。(例如，使用sorted(items)[:N])
- 值得注意的是：`nlargest()`和`nsmallest()`的实际实现会根据使用它们的方式而有所不同，可能会相应做出一些优化措施(比如，当N大小同输入大小很接近时，就会采用排序的方法)

## 1.5 实现优先级队列
